In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os, sys

In [2]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

In [3]:
from scripts.logging import setup_logger
logger = setup_logger()

In [4]:
from scripts.load_data import load_datasets
try:
    train_df, store_df = load_datasets('../Data/train.csv', '../Data/store.csv', logger)
except FileNotFoundError:
    logger.critical("Dataset loading failed due to missing files.")
except Exception as e:
    logger.critical(f"Unexpected error: {e}")

2024-09-24 14:00:13,912 - INFO - Loading datasets.
c:\Users\Coop\Desktop\10 Acadamy\rossmann-sales-prediction-ml\scripts\load_data.py:20: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_file)
2024-09-24 14:00:14,342 - INFO - Datasets loaded successfully.
